In [1]:
import os
import pandas as pd
import json
from PIL import Image
from tqdm import tqdm

def create_yolo_dataset(input_dir, output_dir, csv_files, label_map_file):
    # Create output directories
    os.makedirs(output_dir, exist_ok=True)
    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join(output_dir, split, 'images'), exist_ok=True)
        os.makedirs(os.path.join(output_dir, split, 'labels'), exist_ok=True)

    # Load label map
    with open(label_map_file, 'r') as f:
        label_map = json.load(f)

    # Process each split
    for split, csv_file in csv_files.items():
        df = pd.read_csv(csv_file)

        # Process each image
        for _, row in tqdm(df.iterrows(), total=len(df), desc=f"Processing {split} set"):
            # Get image path and label
            img_path = os.path.join(input_dir, row['Filename'])
            label = row['Label']
            class_name = row['ClassName']

            # Check if the image file exists
            if not os.path.exists(img_path):
                print(f"Warning: {img_path} does not exist. Skipping.")
                continue

            # Open image and convert to RGB (in case it's not)
            img = Image.open(img_path).convert('RGB')
            
            # Create new filename
            new_filename = f"{class_name}_{os.path.basename(img_path).split('.')[0]}.jpg"
            
            # Save image as JPG
            img.save(os.path.join(output_dir, split, 'images', new_filename))

            # Create YOLO format label file
            label_filename = new_filename.replace('.jpg', '.txt')
            with open(os.path.join(output_dir, split, 'labels', label_filename), 'w') as f:
                # YOLO format: <class> <x_center> <y_center> <width> <height>
                # For this dataset, we'll use the entire image as the bounding box
                f.write(f"{label} 0.5 0.5 1.0 1.0\n")

    # Create dataset.yaml file
    yaml_content = f"""
path: {output_dir}
train: train/images
val: val/images
test: test/images

nc: {len(label_map)}
names: {list(label_map.values())}
"""

    with open(os.path.join(output_dir, 'dataset.yaml'), 'w') as f:
        f.write(yaml_content)

# Usage
input_dir = 'C:/Users/IT Department/Downloads/Week2Project/archive/EuroSAT'
output_dir = 'C:/Users/IT Department/Downloads/Week2Project/archive/EuroSAT_yolo'
csv_files = {
    'train': 'C:/Users/IT Department/Downloads/Week2Project/archive/EuroSAT/train.csv',
    'val': 'C:/Users/IT Department/Downloads/Week2Project/archive/EuroSAT/validation.csv',
    'test': 'C:/Users/IT Department/Downloads/Week2Project/archive/EuroSAT/test.csv'
}
label_map_file = 'C:/Users/IT Department/Downloads/Week2Project/archive/EuroSAT/label_map.json'

create_yolo_dataset(input_dir, output_dir, csv_files, label_map_file)

print("Dataset prepared for YOLO in:", output_dir)

Processing test set: 100%|██████████| 2700/2700 [00:08<00:00, 322.28it/s]

Dataset prepared for YOLO in: C:/Users/IT Department/Downloads/Week2Project/archive/EuroSAT_yolo


In [1]:
import os
import torch
from IPython.display import clear_output

# Install YOLOv5
#!pip install -q ultralytics
clear_output()

from ultralytics import YOLO

# Set up paths
data_yaml = 'C:/Users/IT Department/Downloads/Week2Project/archive/EuroSAT_yolo/dataset.yaml'
output_dir = 'C:/Users/IT Department/Downloads/Week2Project/archive/yolov5_eurosat_results'

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Initialize the model
model = YOLO('yolov5su.pt')  # Load a pretrained YOLOv5s model

# Train the model
results = model.train(
    data=data_yaml,
    epochs=20,
    imgsz=64,
    batch=32,
    name='eurosat_yolov5s',
    project='yolov5_eurosat_results',
    exist_ok=True
)

# Evaluate the model on the test set
results = model.val()

print(f"mAP50-95: {results.box.map}")
print(f"mAP50: {results.box.map50}")
print(f"mAP75: {results.box.map75}")

New https://pypi.org/project/ultralytics/8.3.43 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.40  Python-3.12.4 torch-2.5.1+cpu CPU (12th Gen Intel Core(TM) i9-12950HX)
engine\trainer: task=detect, mode=train, model=yolov5su.pt, data=C:/Users/IT Department/Downloads/Week2Project/archive/EuroSAT_yolo/dataset.yaml, epochs=20, time=None, patience=100, batch=32, imgsz=64, save=True, save_period=-1, cache=False, device=None, workers=8, project=yolov5_eurosat_results, name=eurosat_yolov5s, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, ag

train: Scanning C:\Users\IT Department\Downloads\Week2Project\archive\EuroSAT_yolo\train\labels.cache... 18900 images, 0 backgrounds, 0 corrupt: 100%|██████████| 18900/18900 [00:00<?, ?it/s]
val: Scanning C:\Users\IT Department\Downloads\Week2Project\archive\EuroSAT_yolo\val\labels.cache... 5400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5400/5400 [00:00<?, ?it/s]


Plotting labels to yolov5_eurosat_results\eurosat_yolov5s\labels.jpg... 


: 

In [ ]:
# Export the model
model.export(format='onnx')

In [ ]:
test_image = '/kaggle/working/eurosat_yolo/test/images/AnnualCrop_AnnualCrop_1987.jpg'
results = model(test_image)
results

In [ ]:
import os
import torch
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import Counter
import pandas as pd

from IPython.display import clear_output
clear_output()

from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Set up paths
data_yaml = '/kaggle/working/eurosat_yolo/dataset.yaml'
output_dir = '/kaggle/working'
project_name = 'yolov5_eurosat_results'
test_images_dir = '/kaggle/working/eurosat_yolo/test/images'

# Test on all images
true_labels = []
predicted_labels = []
confidences = []

for img_name in os.listdir(test_images_dir):
    img_path = os.path.join(test_images_dir, img_name)
    true_label = img_name.split('_')[0]  # Assuming filename format is "Class_XXXX.jpg"
    
    results = model(img_path)
    
    for r in results:
        boxes = r.boxes
        if len(boxes) > 0:
            # Get the class with highest confidence
            pred_label = model.names[int(boxes.cls[0])]
            confidence = boxes.conf[0].item()
            
            true_labels.append(true_label)
            predicted_labels.append(pred_label)
            confidences.append(confidence)

# Generate classification report
class_report = classification_report(true_labels, predicted_labels)
print("\nClassification Report:")
print(class_report)

# Generate confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.savefig(f"/kaggle/working/{project_name}/confusion_matrix.png")
plt.show()

# Calculate and print accuracy
accuracy = sum([1 for t, p in zip(true_labels, predicted_labels) if t == p]) / len(true_labels)
print(f"\nOverall Accuracy: {accuracy:.4f}")

# Print class-wise accuracy
class_accuracy = Counter()
class_total = Counter()
for true, pred in zip(true_labels, predicted_labels):
    class_total[true] += 1
    if true == pred:
        class_accuracy[true] += 1

print("\nClass-wise Accuracy:")
for cls in class_total:
    print(f"{cls}: {class_accuracy[cls] / class_total[cls]:.4f}")

# Create a DataFrame of predictions
df_predictions = pd.DataFrame({
    'True Label': true_labels,
    'Predicted Label': predicted_labels,
    'Confidence': confidences
})
df_predictions.to_csv(f"/kaggle/working/{project_name}/predictions.csv", index=False)